# Data Wrangling for Sportsbooks Odds Data
We will be gathering our data from multiple sportsbooks from the-odds-api.com, with this data we can begin our data analysis to possibly find arbitrage bets or bets that might significantly favor the bettor
### First we will start with gathering our data from their API

In [434]:
#Import Neccessary modules
import json
import requests
import pandas as pd
api_key = open('OddsAPIKey.txt').read()


In [421]:
#Send a Request for Live Odds of current or upcoming sports
responseodds = requests.get('https://api.the-odds-api.com/v4/sports/upcoming/odds/', params={
    'apiKey': api_key,
    'sport': 'upcoming',
    'regions': 'us',
    'markets': 'h2h',
    'oddsFormat': 'american',
})

In [435]:
# Create a DataFrame from all the data and start to build Datafram fot Data Analysis
df = pd.read_json(responseodds.text)
Titles = list()
for index, row in df.iterrows():
    home = row['home_team']
    away = row['away_team']
    Title = "Home: {} vs Away: {}".format(home,away)
    Titles.append(Title)
bookieslist = list()
for i in range(len(Titles)):
    for ii in range(len(df['bookmakers'][i])):
        bookieslist.append(df['bookmakers'][i][ii]['title'])

cbooks = pd.MultiIndex.from_product([set(bookieslist), ['Home','Away']])
liveodds = pd.DataFrame(index=cbooks, columns=Titles)

In [436]:
#Inputting all the live Data Odds to the right category
for i in range(len(Titles)):
    for ii in range(len(df['bookmakers'][i])):
        liveodds.iloc[:, i][(df['bookmakers'][i][ii]['title'])] = [df['bookmakers'][i][ii]['markets'][0]['outcomes'][0]['price'], df['bookmakers'][i][ii]['markets'][0]['outcomes'][1]['price']]
liveodds = liveodds.drop('PointsBet (US)')
liveodds

/var/folders/wj/x1rb5gs10fb268k5281cx3r00000gn/T/ipykernel_51995/610879412.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  liveodds = liveodds.drop('PointsBet (US)')


Home: Winnipeg Jets vs Away: Chicago Blackhawks  \
Bovada       Home                                             NaN   
             Away                                             NaN   
BetOnline.ag Home                                             NaN   
             Away                                             NaN   
FanDuel      Home                                             NaN   
             Away                                             NaN   
WynnBET      Home                                             NaN   
             Away                                             NaN   
LowVig.ag    Home                                             NaN   
             Away                                             NaN   
DraftKings   Home                                             NaN   
             Away                                             NaN   
BetMGM       Home                                            3300   
             Away                                          -10000   
SuperBook    Home                                             NaN   
             Away                                             NaN   
Caesars      Home                                             NaN   
             Away                                             NaN   
BetRivers    Home                                             NaN   
             Away                                             NaN   
BetUS        Home                                             NaN   
             Away                                             NaN   
Unibet       Home                                             NaN   
             Away                                             NaN   
MyBookie.ag  Home                                             NaN   
             Away                                             NaN   
TwinSpires   Home                                             NaN   
             Away                                             NaN   

                  Home: UNLV Rebels vs Away: Boise State Broncos  \
Bovada       Home                                           -900   
             Away                                            500   
BetOnline.ag Home                                            NaN   
             Away                                            NaN   
FanDuel      Home                                          -1100   
             Away                                            600   
WynnBET      Home                                            NaN   
             Away                                            NaN   
LowVig.ag    Home                                            NaN   
             Away                                            NaN   
DraftKings   Home                                           -750   
             Away                                            470   
BetMGM       Home                                           -700   
             Away                                            475   
SuperBook    Home                                            NaN   
             Away                                            NaN   
Caesars      Home                                          -1000   
             Away                                            600   
BetRivers    Home                                            NaN   
             Away                                            NaN   
BetUS        Home                                            NaN   
             Away                                            NaN   
Unibet       Home                                          -1115   
             Away                                            650   
MyBookie.ag  Home                                           -600   
             Away                                            350   
TwinSpires   Home                                          -1115   
             Away                                            650   

                  Home: Coastal Carolina Ch

In [437]:
## We can then grab and analyze our data as we please, first we seperate Home and Away odds and find the best odds
# homeodds = liveodds.iloc[:, 0].dropna().reset_index(1)[liveodds.iloc[:, 0].dropna().reset_index(1)['level_1'] == 'Home']
# awayodds = liveodds.iloc[:, 0].dropna().reset_index(1)[liveodds.iloc[:, 0].dropna().reset_index(1)['level_1'] == 'Away']
## Lets create a list for discrepancies between all odds
# Homedisc = homeodds.iloc[:, 1].max() - homeodds.iloc[:, 1].min()
# Awaydisc = awayodds.iloc[:, 1].max() - awayodds.iloc[:, 1].min()
# disclist = list()
# disclist.append([Homedisc, Awaydisc])

In [438]:
#Now list do this for all Odds for upcoming or Live Events
disclist = list()
for i in range(len(Titles)):
    homeodds = liveodds.iloc[:, i].dropna().reset_index(1)[liveodds.iloc[:, i].dropna().reset_index(1)['level_1'] == 'Home']
    awayodds = liveodds.iloc[:, i].dropna().reset_index(1)[liveodds.iloc[:, i].dropna().reset_index(1)['level_1'] == 'Away']
    Homedisc = homeodds.iloc[:, 1].max() - homeodds.iloc[:, 1].min()
    Awaydisc = awayodds.iloc[:, 1].max() - awayodds.iloc[:, 1].min()
    disclist.append([Homedisc, Awaydisc])
discframe = pd.DataFrame(index=Titles, columns=['Home', 'Away'], data=disclist)

In [439]:
# So this list right here gives us a list of the current games going on with the biggest discrepancies between
# bookies, this gives a very good look into games with possibly large arbitrage opportunities
discframe.dropna().sort_values(by=['Home'], ascending=False)

,Home,Away
Home: Casa Pia vs Away: Portimonense,19000.0,12600.0
Home: Tyrone McKenna vs Away: Lewis Crocker,5000.0,300.0
Home: Stony Brook Seawolves vs Away: Wagner Seahawks,2015.0,6165.0
Home: Vanderbilt Commodores vs Away: Alabama A&M Bulldogs,1500.0,16000.0
Home: Corinthians vs Away: Internacional,1480.0,710.0
Home: Palestino vs Away: Curicó Unido,1300.0,315.0
Home: Audax Italiano vs Away: Magallanes,1100.0,400.0
Home: UCF Knights vs Away: Lipscomb Bisons,1000.0,6000.0
Home: Kentucky Wildcats vs Away: UNC Wilmington Seahawks,1000.0,300.0
Home: Ihor Potieria vs Away: Rodolfo Bellato,850.0,670.0


In [442]:
#Lets print off the top 10 games with the largest discrepancies and look at the numbers, using this list we can
#find potential arbitrage opportunities and also find the most profitable bookie for any current live odds of a game
from IPython.display import display, HTML
for i in range(4):
    display(liveodds[[discframe.dropna().sort_values(by=['Home'], ascending=False).index[i]]].dropna())
    display(liveodds[[discframe.dropna().sort_values(by=['Away'], ascending=False).index[i]]].dropna()) 

Home: Casa Pia vs Away: Portimonense
DraftKings  Home                               -20000
            Away                                 5000
BetMGM      Home                               -10000
            Away                                15000
MyBookie.ag Home                                -1000
            Away                                 2400

Home: Loyola (Chi) Ramblers vs Away: Harvard Crimson
DraftKings Home                                               4000  
           Away                                            -100000  
BetMGM     Home                                               3300  
           Away                                             -10000

Home: Tyrone McKenna vs Away: Lewis Crocker
FanDuel    Home                                       -8000
           Away                                        1600
DraftKings Home                                       -5000
           Away                                        1300
BetMGM     Home                                      -10000
           Away                                        1600

Home: Vanderbilt Commodores vs Away: Alabama A&M Bulldogs
Bovada     Home                                               1000       
           Away                                              -4000       
FanDuel    Home                                               2500       
           Away                                             -20000       
DraftKings Home                                               2200       
           Away                                             -10000       
BetMGM     Home                                               1600       
           Away                                             -10000

Home: Stony Brook Seawolves vs Away: Wagner Seahawks
Bovada     Home                                               -150  
           Away                                                115  
DraftKings Home                                               -215  
           Away                                                165  
Caesars    Home                                               1800  
           Away                                              -6000

Home: Casa Pia vs Away: Portimonense
DraftKings  Home                               -20000
            Away                                 5000
BetMGM      Home                               -10000
            Away                                15000
MyBookie.ag Home                                -1000
            Away                                 2400

Home: Vanderbilt Commodores vs Away: Alabama A&M Bulldogs
Bovada     Home                                               1000       
           Away                                              -4000       
FanDuel    Home                                               2500       
           Away                                             -20000       
DraftKings Home                                               2200       
           Away                                             -10000       
BetMGM     Home                                               1600       
           Away                                             -10000

Home: Stony Brook Seawolves vs Away: Wagner Seahawks
Bovada     Home                                               -150  
           Away                                                115  
DraftKings Home                                               -215  
           Away                                                165  
Caesars    Home                                               1800  
           Away                                              -6000